# Is er een relatie tussen klimaat verandering en het vertrouwen in de overheid?

In [1]:
import csv
from io import StringIO

import matplotlib.pyplot as plt
import folium
import ipywidgets as widgets
from IPython.display import display
import datetime
import findspark
findspark.init()
import pyspark
sc= pyspark.SparkContext(appName='myAppName')
sc_sql=pyspark.SQLContext(sc)

your 131072x1 screen size is bogus. expect trouble
23/12/06 13:54:28 WARN Utils: Your hostname, LAPTOP-7K3D8E7S resolves to a loopback address: 127.0.1.1; using 172.22.10.172 instead (on interface eth0)
23/12/06 13:54:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/06 13:54:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/06 13:54:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/home/loic/miniconda3/envs/data_presentation/lib/python3.12/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Data exploratiie


Ik lees de data in als rdd en bekijk hoe de records eruit zien

In [2]:
rdd_greenhouse_emission_raw=sc.textFile('data/climate_trustInGou/greenhouse_emission.csv')
rdd_trust_in_goverment_raw=sc.textFile('data/climate_trustInGou/DP_LIVE_22112023123634891.csv')

de greenhouse_emission dataset richt zich op de uitstoot van broeikasgassen, met name koolstofdioxide (CO2) en methaan (CH4), en geeft een gedetailleerde uitsplitsing voor verschillende landen. De dataset is voor het laatst geupdate in 2022. Daarom ga ik er van uit dat deze data toepasselijk is voor dat jaar. Deze veronderstelling is belangrijk voor later wanneer we de datasets gaan samenvoegen

In [3]:
rdd_greenhouse_emission_raw.take(7)


['country,total_co2,co2_per_capita,total_methane,methane_per_capita',
 'Qatar,91.97 m t,31.38 t,21.45 m t,7.32 t',
 'Bahrain,33.26 m t,19.02 t,17.41 m t,9.96 t',
 'Kuwait,92.65 m t,21.40 t,24.71 m t,5.71 t',
 'Turkmenistan,72.87 m t,11.91 t,80.48 m t,13.15 t',
 'United Arab Emirates,188.86 m t,18.90 t,49.80 m t,4.98 t',
 'Grenada,"330,000.00 t",2.92 t,2.04 m t,18.05 t']

de DP_LIVE dataset hernoem ik naar trust_in_goverment omdat dit laatste de data beschrijft.De dataset geeft voor verschillende landen procentueel het vertrouwen in de overheid weer voor elk jaar vanaf 2006 tot 2022

In [4]:
rdd_trust_in_goverment_raw.take(5)

['"LOCATION","INDICATOR","SUBJECT","MEASURE","FREQUENCY","TIME","Value","Flag Codes"',
 '"AUS","TRUSTGOV","TOT","PC","A","2006",53.206332837474,',
 '"AUS","TRUSTGOV","TOT","PC","A","2007",52.6932810746771,',
 '"AUS","TRUSTGOV","TOT","PC","A","2008",64.4675424053232,',
 '"AUS","TRUSTGOV","TOT","PC","A","2010",60.9752918764891,']

## Data manipulatie

### greenhouse emmision


De greenhouse emission dataset heeft 5 kolommen,een header en in sommige values staan kommas. Deze kommas zorgen voor foute inlezing als ik de split functie gebruik

In [5]:
rdd_greenhouse_emission_split = rdd_greenhouse_emission_raw.map(
    lambda line: line.split(',')
).filter(lambda x: x[0] != 'country').map(
    lambda x: ((x[0].strip("''")), x[1], x[2], x[3], x[4])
)

rdd_greenhouse_emission_split.take(6)

[('Qatar', '91.97 m t', '31.38 t', '21.45 m t', '7.32 t'),
 ('Bahrain', '33.26 m t', '19.02 t', '17.41 m t', '9.96 t'),
 ('Kuwait', '92.65 m t', '21.40 t', '24.71 m t', '5.71 t'),
 ('Turkmenistan', '72.87 m t', '11.91 t', '80.48 m t', '13.15 t'),
 ('United Arab Emirates', '188.86 m t', '18.90 t', '49.80 m t', '4.98 t'),
 ('Grenada', '"330', '000.00 t"', '2.92 t', '2.04 m t')]

hierboven zien we dus dat de waarde 330, 000.00 t fout wordt ingelezen als 2 kolommen. dit zorgt er ook voor dat de laatse waarde uit die rij niet wordt ingelezen. Na grondig bekijken van rdd_greenhouse_emission_raw merk ik dat de values waar er een komma  wordt gebruikt voor leesbaarheid van het getal de enigste values zijn met nog dubbele quotes (") rond. 

Als vervanging van de split functie gebruik ik de csv.reader functie in samenwerking met de StringIO klasse van de io library. Deze klasse zorgt ervoor dat er een in-memory file-like object van de string wordt gemaakt. Dat object kan dan ingelezen worden door de csv.read functie.

In [6]:
def remove_quotes_and_commas_numeric(value):
    # Remove quotes
    value_without_quotes = value.replace('"', '')

    # Remove commas within numbers
    value_without_commas = ''.join(c for c in value_without_quotes if c.isnumeric() or c == '.' or c == 't' or c == 'm' or c == 'b' or c == 'n')

    return value_without_commas

In [7]:
rdd_greenhouse_emission_rmQC = rdd_greenhouse_emission_raw.map(
    lambda line: list(csv.reader(StringIO(line)))[0]
).filter(lambda x: x[0] != 'country').map(
    lambda row: (
        row[0],
        remove_quotes_and_commas_numeric(row[1]),
        remove_quotes_and_commas_numeric(row[2]),
        remove_quotes_and_commas_numeric(row[3]),
        remove_quotes_and_commas_numeric(row[4])
    )
)

rdd_greenhouse_emission_rmQC.take(14)


[('Qatar', '91.97mt', '31.38t', '21.45mt', '7.32t'),
 ('Bahrain', '33.26mt', '19.02t', '17.41mt', '9.96t'),
 ('Kuwait', '92.65mt', '21.40t', '24.71mt', '5.71t'),
 ('Turkmenistan', '72.87mt', '11.91t', '80.48mt', '13.15t'),
 ('United Arab Emirates', '188.86mt', '18.90t', '49.80mt', '4.98t'),
 ('Grenada', '330000.00t', '2.92t', '2.04mt', '18.05t'),
 ('New Caledonia', '5.33mt', '19.54t', '214726.32t', '0.79t'),
 ('Australia', '386.53mt', '15.02t', '126.07mt', '4.90t'),
 ('Brunei', '6.99mt', '15.83t', '1.59mt', '3.60t'),
 ('Gibraltar', '634391.00t', '18.83t', '6604.06t', '0.20t'),
 ('Saudi Arabia', '523.78mt', '14.82t', '110.25mt', '3.12t'),
 ('Canada', '580.21mt', '15.17t', '101.70mt', '2.66t'),
 ('United States', '4.82bnt', '14.52t', '744.51mt', '2.24t'),
 ('Russia', '1.70bnt', '11.88t', '684.30mt', '4.77t')]

 Om goed te kunnen werken met deze dataset is het van groot belang om alle waardes om te zetten naar dezelfde eenheid.  de kleinste eenheid is t dus laten we alle waardes omzetten naar die eenheid.

In [8]:
def to_t(value):
    # Extract numeric part and unit
    numeric_part = ''.join(c for c in value if c.isnumeric() or c == '.')
    unit_part = ''.join(c for c in value if c.isalpha())

    # Convert numeric part to float
    numeric_value = float(numeric_part.replace(',', ''))

    # Determine the multiplier based on the unit
    if unit_part == 't':
        return numeric_value
    elif unit_part == 'mt':
        return numeric_value * (10 ** 6)
    elif unit_part == 'bnt':
        return numeric_value * (10 ** 9)

In [9]:
rdd_greenhouse_emission_in_t = rdd_greenhouse_emission_rmQC.map(
    lambda row: (
        row[0],
        to_t(row[1]),
        to_t(row[2]),
        to_t(row[3]),
        to_t(row[4])
    ))

rdd_greenhouse_emission_in_t.take(14)


[('Qatar', 91970000.0, 31.38, 21450000.0, 7.32),
 ('Bahrain', 33259999.999999996, 19.02, 17410000.0, 9.96),
 ('Kuwait', 92650000.0, 21.4, 24710000.0, 5.71),
 ('Turkmenistan', 72870000.0, 11.91, 80480000.0, 13.15),
 ('United Arab Emirates', 188860000.0, 18.9, 49800000.0, 4.98),
 ('Grenada', 330000.0, 2.92, 2040000.0, 18.05),
 ('New Caledonia', 5330000.0, 19.54, 214726.32, 0.79),
 ('Australia', 386530000.0, 15.02, 126070000.0, 4.9),
 ('Brunei', 6990000.0, 15.83, 1590000.0, 3.6),
 ('Gibraltar', 634391.0, 18.83, 6604.06, 0.2),
 ('Saudi Arabia', 523780000.0, 14.82, 110250000.0, 3.12),
 ('Canada', 580210000.0, 15.17, 101700000.0, 2.66),
 ('United States', 4820000000.0, 14.52, 744510000.0, 2.24),
 ('Russia', 1700000000.0, 11.88, 684300000.0, 4.77)]

We bekomen een rdd met 5 kolommen. De eerste kolom bevat een string met de naam van het land, de anderen co2 en methaan waardes in de eenheid ton

### trust in goverment

deze dataset split zijn rows op adhv ''. naast de komma die de waardes onderscheiden, staan er rond iedere waarde(excl de laatste) ook nog eens "".

In [10]:
rdd_trust_in_goverment_manipulatie = rdd_trust_in_goverment_raw.map(
    lambda line: line.split(',')
    ).filter(
        lambda x: x[0] != '"LOCATION"'
        ).map(
            lambda x: (x[0].strip('"'),int(x[5].strip('"')),float(x[6].strip(',')))
            )
rdd_trust_in_goverment_manipulatie.take(5)

[('AUS', 2006, 53.206332837474),
 ('AUS', 2007, 52.6932810746771),
 ('AUS', 2008, 64.4675424053232),
 ('AUS', 2010, 60.9752918764891),
 ('AUS', 2011, 53.0786719332192)]

Nu dat we de benodigde kolommen uit de dataset hebben gehaald en mooi gezet is het tijd om de onnodige datarows te verwijderen. Zoals eerder aangehaald kan ik enkel de rijen met jaartal 2022 gebruiken.

In [11]:
rdd_trust_in_goverment_2022=rdd_trust_in_goverment_manipulatie.filter(
    lambda x: x[1] == 2022
).map(
    lambda x: (x[0], x[2])
    )
rdd_trust_in_goverment_2022.take(5)

[('AUS', 49.9118816780199),
 ('BEL', 57.2408419337754),
 ('CAN', 50.6668033321548),
 ('CZE', 34.0826747537788),
 ('DNK', 63.5359926406944)]

### Landen

voor de ondezrzoeksvraag te kunnen beantwoorden hebben we enkel baad aan landen die in beide datasets voorkomen. Doordat in beide datasets de landen anders worden genoteerd vraagt dit extra aandacht 

Na enig zoekwerk heb ik de dataset continents2 gevonden. Deze data bevat alle landen met hun ISO 3166-1 alpha-3 notatie. Deze dataset ga ik gebruiken om de country kolom in de trust dataset om te zetten naar de volledige naam.

In [12]:
continents2_raw = sc.textFile("data/climate_trustInGou/continents2.csv")
continents2_raw.take(5)

['name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code',
 'Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,,142,34,',
 'Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,,150,154,',
 'Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,,150,39,',
 'Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,,2,15,']

we hebben enkel nood aan de kolommen name en alpha-3

In [13]:
continents2_manipulatie=continents2_raw.map(
    lambda line: line.split(',')
).filter(
    lambda x: x[0] !='name'
    ).map(
        lambda x: (x[0],x[2])
    )

continents2_manipulatie.take(5)

[('Afghanistan', 'AFG'),
 ('Åland Islands', 'ALA'),
 ('Albania', 'ALB'),
 ('Algeria', 'DZA'),
 ('American Samoa', 'ASM')]

Deze dataset moeten we filteren op landen die voorkomen in voorgaande datatasets.

In [14]:
greenhouse_countries = rdd_greenhouse_emission_in_t.map(lambda x: x[0]).collect()

continents2_in_emission = continents2_manipulatie.filter(lambda x: x[0] in greenhouse_countries)

goverment_countries = rdd_trust_in_goverment_2022.map(lambda x: x[0]).collect()

continents_in_both  = continents2_in_emission.filter(lambda x: x[1] in goverment_countries)

In [15]:
continents_list = continents_in_both.collect()

for row in continents_list:
    print(row)

('Australia', 'AUS')
('Belgium', 'BEL')
('Brazil', 'BRA')
('Canada', 'CAN')
('Chile', 'CHL')
('Colombia', 'COL')
('Costa Rica', 'CRI')
('Denmark', 'DNK')
('Estonia', 'EST')
('Finland', 'FIN')
('Germany', 'DEU')
('Greece', 'GRC')
('Hungary', 'HUN')
('Iceland', 'ISL')
('Japan', 'JPN')
('Mexico', 'MEX')
('Netherlands', 'NLD')
('New Zealand', 'NZL')
('Norway', 'NOR')
('Poland', 'POL')
('Portugal', 'PRT')
('Sweden', 'SWE')
('United States', 'USA')


er zijn dus maar 23 landen waarvan we zowel de uitlaatgasdata en vertrouwingsdata hebben

ik wil uitijndelijk op een wereld kaart het vertrouwen in en de hoeveelheid co2 per hoofd van een land visualiseren. Om dit te verwezenlmijken heb ik de lengte- en breedtegraad van elk land nodig

In [16]:
rdd_lat_long_raw = sc.textFile('data/climate_trustInGou/world_country_and_usa_states_latitude_and_longitude_values.csv')
rdd_lat_long_raw.take(5)

['country_code,latitude,longitude,country,usa_state_code,usa_state_latitude,usa_state_longitude,usa_state',
 'AD,42.546245,1.601554,Andorra,AK,63.588753,-154.493062,Alaska',
 'AE,23.424076,53.847818,United Arab Emirates,AL,32.318231,-86.902298,Alabama',
 'AF,33.93911,67.709953,Afghanistan,AR,35.20105,-91.831833,Arkansas',
 'AG,17.060816,-61.796428,Antigua and Barbuda,AZ,34.048928,-111.093731,Arizona']

we hebben enkel de kolommen country,latitude en longitude nodig. in de csv file missen er voor sommige landen lengte- en breedtegraad, deze kunnen dus niet gebruikt worden.

In [17]:
rdd_country_lat_long = rdd_lat_long_raw.map(
    lambda line: line.split(',')
).filter(lambda x: x[0] != 'country_code' and x[1] != '' and x[2] != '').map(
    lambda x: (x[3], float(x[1]), float(x[2]))
)

rdd_country_lat_long.take(245)


[('Andorra', 42.546245, 1.601554),
 ('United Arab Emirates', 23.424076, 53.847818),
 ('Afghanistan', 33.93911, 67.709953),
 ('Antigua and Barbuda', 17.060816, -61.796428),
 ('Anguilla', 18.220554, -63.068615),
 ('Albania', 41.153332, 20.168331),
 ('Armenia', 40.069099, 45.038189),
 ('Netherlands Antilles', 12.226079, -69.060087),
 ('Angola', -11.202692, 17.873887),
 ('Antarctica', -75.250973, -0.071389),
 ('Argentina', -38.416097, -63.616672),
 ('American Samoa', -14.270972, -170.132217),
 ('Austria', 47.516231, 14.550072),
 ('Australia', -25.274398, 133.775136),
 ('Aruba', 12.52111, -69.968338),
 ('Azerbaijan', 40.143105, 47.576927),
 ('Bosnia and Herzegovina', 43.915886, 17.679076),
 ('Barbados', 13.193887, -59.543198),
 ('Bangladesh', 23.684994, 90.356331),
 ('Belgium', 50.503887, 4.469936),
 ('Burkina Faso', 12.238333, -1.561593),
 ('Bulgaria', 42.733883, 25.48583),
 ('Bahrain', 25.930414, 50.637772),
 ('Burundi', -3.373056, 29.918886),
 ('Benin', 9.30769, 2.315834),
 ('Bermuda', 3

### samenvoeging van de data

om de data samen te voegen filter ik de rdd_trust_in_goverment_2022, de rdd_greenhouse_emission_in_t en de rdd_long_lat_country op de continents_list

In [18]:
# Create lists from the first and second columns of continents_in_both
first_column_list = continents_in_both.map(lambda x: x[0]).collect()
second_column_list = continents_in_both.map(lambda x: x[1]).collect()

# Filter the trust RDD based on the second column
filtered_trust_rdd = rdd_trust_in_goverment_2022.filter(lambda x: x[0] in second_column_list)

# Filter the emission RDD based on the first column
filtered_emission_rdd = rdd_greenhouse_emission_in_t.filter(lambda x: x[0] in first_column_list)

# Filter the country RDD based on the first column
filtered_country_rdd = rdd_country_lat_long.filter(lambda x: x[0] in first_column_list)
# Print the results
print("Filtered Trust RDD:")
print(filtered_trust_rdd.collect())

print("\nFiltered Emission RDD:")
print(filtered_emission_rdd.collect())

print("\nFiltered country RDD:")
print(filtered_country_rdd.collect())

Filtered Trust RDD:
[('AUS', 49.9118816780199), ('BEL', 57.2408419337754), ('CAN', 50.6668033321548), ('DNK', 63.5359926406944), ('FIN', 77.5449915345894), ('DEU', 60.7603777839133), ('GRC', 25.6266144139102), ('HUN', 44.1911725509736), ('ISL', 51.5324987127802), ('JPN', 43.1462692986385), ('MEX', 52.8816455058393), ('NLD', 47.2347701581325), ('NZL', 51.3921513069705), ('NOR', 63.6077672008499), ('POL', 34.1607151853422), ('PRT', 58.92422646215), ('SWE', 68.7863033630634), ('USA', 31.024155522486), ('BRA', 39.1633431519861), ('CHL', 28.7241784844887), ('COL', 29.6825079323217), ('CRI', 60.0127479827011), ('EST', 50.8495859391006)]

Filtered Emission RDD:
[('Australia', 386530000.0, 15.02, 126070000.0, 4.9), ('Canada', 580210000.0, 15.17, 101700000.0, 2.66), ('United States', 4820000000.0, 14.52, 744510000.0, 2.24), ('New Zealand', 34010000.0, 6.64, 32520000.000000004, 6.35), ('Netherlands', 146340000.0, 8.35, 15540000.0, 0.89), ('Japan', 1080000000.0, 8.61, 27380000.0, 0.22), ('Belgium

na deze filtering is het tijd om de landkolom in de trust rdd om te zetten naar de andere notatie. Ik kies voor het omzetten van deze notatie omdat ik het makkelijker vindt om landen voluit te lezen en omdat de andere 2 rdds dezelfde notatie hebben. Het omzetten van 2 kolommen duurt langer dan 1 kolom omzetten

In [19]:

def change_to_full_name(row):
    code,trust = row
    for _ in continents_list:
        if code == _[1]:
            name=_[0]
            break
    
    return name, trust
trust_rdd_full_names = filtered_trust_rdd.map(
        lambda x: change_to_full_name(x)
    )

trust_rdd_full_names.take(5)

[('Australia', 49.9118816780199),
 ('Belgium', 57.2408419337754),
 ('Canada', 50.6668033321548),
 ('Denmark', 63.5359926406944),
 ('Finland', 77.5449915345894)]

nu dat alle rdd's een kolom bevatten waar de namen van de landen voluit zijn geschreven kan ik ze bij elkaar toevoegen

In [20]:
list_trust = trust_rdd_full_names.collect()
list_long_lat = filtered_country_rdd.collect()
def add_trust_score_and_lat_long(row):
    name,v1,v2,v3,v4 = row
    v5 = next((trust for country, trust in list_trust if country == name), None)
    #bovenstaande is een oneliner voor:
    #for _ in list_trust:
    #    if name == _[0]:
    #        v5=_[1]
    #        break
    for _ in list_long_lat:
        if name == _[0]:
            v6=_[1]
            v7=_[2]
    return name,v1,v2,v3,v4,v5,v6,v7

In [21]:


final_rdd = filtered_emission_rdd.map(
    lambda x: add_trust_score_and_lat_long(x)
)

final_rdd.take(23)


[('Australia',
  386530000.0,
  15.02,
  126070000.0,
  4.9,
  49.9118816780199,
  -25.274398,
  133.775136),
 ('Canada',
  580210000.0,
  15.17,
  101700000.0,
  2.66,
  50.6668033321548,
  56.130366,
  -106.346771),
 ('United States',
  4820000000.0,
  14.52,
  744510000.0,
  2.24,
  31.024155522486,
  37.09024,
  -95.712891),
 ('New Zealand',
  34010000.0,
  6.64,
  32520000.000000004,
  6.35,
  51.3921513069705,
  -40.900557,
  174.885971),
 ('Netherlands',
  146340000.0,
  8.35,
  15540000.0,
  0.89,
  47.2347701581325,
  52.132633,
  5.291266),
 ('Japan',
  1080000000.0,
  8.61,
  27380000.0,
  0.22,
  43.1462692986385,
  36.204824,
  138.252924),
 ('Belgium',
  93010000.0,
  8.03,
  8080000.0,
  0.7,
  57.2408419337754,
  50.503887,
  4.469936),
 ('Poland',
  295130000.0,
  7.81,
  29000000.0,
  0.77,
  34.1607151853422,
  51.919438,
  19.145136),
 ('Germany',
  657400000.0,
  7.91,
  48710000.0,
  0.59,
  60.7603777839133,
  51.165691,
  10.451526),
 ('Estonia',
  10180000.0,
 

de uiteindelijke rdd heeft 8 kolommen enn 23 rijen. de kolommen van links naar rechts luiden: naam van land, total_co2, co2_per_capita, total_methane, methane_per_capita en trust_value;latitude, longitude

## Data presentation

In [76]:
world_map=folium.Map(location=[0,0],zoom_start=2)


final_list = final_rdd.collect()

# Create a map centered at [0, 0] with a zoom level of 2
total_co2_world_map= world_map

# Create a linear colormap from blue to red
colormap = folium.LinearColormap(colors=['yellow', 'blue'], vmin=0, vmax=100)

# Iterate over the list and add circle markers to the map
for country, total_co2, co2_per_capita, total_methane, methane_per_capita, trust_value, lat, long in final_list:
    color = colormap(trust_value)

    # Create a circle marker for each country
    folium.CircleMarker(
        location=[lat, long],
        radius=10 + total_co2/(10**7.75),
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"{country}<br>total CO2: {total_co2}t<br>Trust value: {trust_value}%"
    ).add_to(total_co2_world_map)
    
# Display the map
total_co2_world_map


In [77]:
final_list = final_rdd.collect()

# Create a map centered at [0, 0] with a zoom level of 2
co2_per_capita_world_map= world_map


# Iterate over the list and add circle markers to the map
for country, total_co2, co2_per_capita, total_methane, methane_per_capita, trust_value, lat, long in final_list:
    color=colormap(trust_value)

    # Create a circle marker for each country
    folium.CircleMarker(
        location=[lat, long],
        radius=10+co2_per_capita,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"{country}<br>CO2 per capita: {co2_per_capita}t<br>Trust value: {trust_value}%"
    ).add_to(co2_per_capita_world_map)
    
# Display the map
co2_per_capita_world_map


In [79]:
final_list = final_rdd.collect()

# Create a map centered at [0, 0] with a zoom level of 2
total_methane_world_map= world_map

# Iterate over the list and add circle markers to the map
for country, total_co2, co2_per_capita, total_methane, methane_per_capita, trust_value, lat, long in final_list:
    color=colormap(trust_value)

    # Create a circle marker for each country
    folium.CircleMarker(
        location=[lat, long],
        radius=10+total_methane/(10**6.50),
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"{country}<br>total methane: {total_methane}t<br>Trust value: {trust_value}%"
    ).add_to(total_methane_world_map)
    
# Display the map
total_methane_world_map


In [78]:
final_list = final_rdd.collect()

# Create a map centered at [0, 0] with a zoom level of 2
methane_per_capita_world_map= world_map

# Iterate over the list and add circle markers to the map
for country, total_co2, co2_per_capita, total_methane, methane_per_capita, trust_value, lat, long in final_list:
    color=colormap(trust_value)

    # Create a circle marker for each country
    folium.CircleMarker(
        location=[lat, long],
        radius=10+methane_per_capita*(10),
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"{country}<br>Methane per capita: {methane_per_capita} t<br>Trust value: {trust_value}%"
    ).add_to(methane_per_capita_world_map)
    
# Display the map
methane_per_capita_world_map
